In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

import scipy.sparse as sparse
import random
# import implicit
# from implicit.als import AlternatingLeastSquares as ALS

%cd /home/user_3/medistream-recsys/Script
from preprocessing import drop_columns,dict_to_column,dict_to_set,set_to_column,key_to_element

pd.set_option('display.max_rows', 300)
pd.set_option('display.max_columns', 100)

import warnings
warnings.filterwarnings("ignore")

/home/user_3/medistream-recsys/Script


In [2]:
df1 = pd.read_json('/fastcampus-data/select_column_version_3.json')
df = df1.copy()

In [21]:
start_date = "2022-05-12"

# paid orders만 가져오기
df['date_paid'] = pd.to_datetime(df['date_paid'])
df_only_paid = df[~df['date_paid'].isna()]

# 3개월치 데이터만 가져오기
df_date = df_only_paid[df_only_paid['date_paid'] > start_date]

# 취소 안된 것만 가져오기
complete_df = df_date[(df_date['paid'] == True) & (df_date['cancelled']==False)]
complete_df = complete_df.reset_index(drop=True)

# 도서 카테고리만 가져오기
df_book = complete_df[complete_df['name'] == '도서']
df_book = df_book.sort_values(by='date_paid')


# train test split

In [30]:
from datetime import datetime, timedelta

date = df_book['date_paid'].max()-timedelta(weeks=3)
train_before_preprocess = df_book[df_book['date_paid'] < date]
test_before_preprocess = df_book[df_book['date_paid'] >= date]

# train 3개 이상 구매 기록유저에 대한 학습 진행
train_drop_row_index = train_before_preprocess['customer_id'].value_counts()[train_before_preprocess['customer_id'].value_counts()>=1].index
train = train_before_preprocess[train_before_preprocess['customer_id'].isin(train_drop_row_index)]

In [31]:
# 전처리할 test index 확인
drop_row_index = test_before_preprocess['customer_id'].value_counts()[test_before_preprocess['customer_id'].value_counts()>2].index

# 최근 구매 3개 기록을 test 사용
test_list = []
for idx in drop_row_index:
    test_list.append(test_before_preprocess[test_before_preprocess['customer_id']==idx].iloc[:])
test = pd.concat(test_list)

# train 있는 유저에 대해서만 진행
test = test[test['customer_id'].isin(train['customer_id'])]

In [32]:
train.date_paid = pd.to_datetime(train.date_paid)

In [33]:
# date_paid에 날짜만 남기기
train.date_paid = train.date_paid.apply(lambda x: x.date)

# train test eda

### 전처리 전후 비교

In [34]:
print('train 전처리 전:',len(train_before_preprocess), 'train 전처리 후:',len(train))

train 전처리 전: 5042 train 전처리 후: 5042


In [35]:
print('test 전처리 전:',len(test_before_preprocess), 'test 전처리 후:',len(test))

test 전처리 전: 889 test 전처리 후: 116


### user 수 비교

In [36]:
print('train 유저 수:',len(train.customer_id.unique()))

train 유저 수: 2323


In [37]:
print('test 유저 수:',len(test.customer_id.unique()))

test 유저 수: 29


### item 개수 비교

In [38]:
print('train 아이템 수 :',len(set(train.product_ids)), 'test 아이템 수 :',len(set(test.product_ids)))

train 아이템 수 : 243 test 아이템 수 : 67


In [39]:
print('train 만 있는 아이템 수:',  len(set(train.product_ids)-set(test.product_ids)))

train 만 있는 아이템 수: 180


In [52]:
print('test 만 있는 아이템 수:', len(set(test.product_ids) - set(train.product_ids)))

test 만 있는 아이템 수: 4


In [53]:
print(train.shape)
print(test.shape)


(5042, 16)
(116, 16)


# copy kaggle example

# Load Transactions, Reduce Memory

In [40]:
train = train[['date_paid','customer_id','name_x','product_ids']]

In [41]:
tmp = train.groupby('customer_id').date_paid.max().reset_index()
tmp.columns = ['customer_id','max_dat']
train = train.merge(tmp,on=['customer_id'],how='left')
train['diff_dat'] = (train.max_dat - train.date_paid).dt.days

# 다른 날짜를 6보다 작은 것으로 하는건 최신의 정보를 반영하기 위함인가?
train = train.loc[train['diff_dat']<=6]
print('Train shape:',train.shape)

Train shape: (3222, 6)


In [ ]:
train.diff_dat.unique()

# Apriori Transformation:

In [56]:
#Apriori treats duplicates as a single occurence
# 한 유저당 구매한 제품들의 unique -> 중복을 없앰.
train_apri = train.groupby('customer_id')['name_x'].unique().reset_index()
train_apri[:30]

,customer_id,name_x
0,5d60cab24e77525ec5ca13d5,[숲을 보는 요통치료]
1,5d60cb754e77525ec5ca13d9,[흔히보는 정형외과 외래진료 가이드북]
2,5d60cc294e77525ec5ca13e0,[약침의 정석 –통증편]
3,5d60ccd84e77525ec5ca13f2,[증보운곡본초학]
4,5d60cce84e77525ec5ca13f3,[보험한약 브런치 the # 2판 개정판]
5,5d60cd1b4e77525ec5ca13fd,"[윤상훈·권병조의 알짜 근육학, 동씨침법입문 2판]"
6,5d60cd414e77525ec5ca1401,[초음파 유도하 침 시술 가이드북]
7,5d60cd634e77525ec5ca1402,[뇌의학의 첫걸음]
8,5d60ce694e77525ec5ca140e,[趙紹琴(조소금) 내과학]
9,5d60cea34e77525ec5ca1413,[비만문답]


In [57]:
from mlxtend.preprocessing import TransactionEncoder

In [58]:
pd.set_option('display.max_colwidth', -1)
transactions = [a[1]['name_x'].tolist() for a in list(train.groupby(['customer_id','date_paid']))]

In [59]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)

# orders_1hot

In [60]:
orders_1hot = pd.DataFrame(te_ary, columns=te.columns_)
pf = orders_1hot.describe()

In [61]:
f = pf.iloc[0]-pf.iloc[3] # count - frequent
a = f.tolist()
b = list(f.index)
item = pd.DataFrame([[a[r],b[r]]for r in range(len(a))], columns=['Count','Item'])
item = item.sort_values(['Count'], ascending=False).head(10)
print("MOST POPULAR")
item

MOST POPULAR


,Count,Item
78,432,비만문답
190,363,플로차트 정형외과 진단
132,188,윤상훈·권병조의 알짜 근육학
217,145,흔히보는 정형외과 외래진료 가이드북
122,117,외래에서 꼭 알아야 할 통증증후군 137가지
216,110,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼
215,106,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
168,86,초음파 유도하 침 시술 가이드북
84,82,사암침의 해석과 임상
146,72,임상 한의사를 위한 기본 한약처방 강의 2판


In [62]:
from mlxtend.frequent_patterns import apriori

In [79]:
is_ap = apriori(orders_1hot, min_support=0.01, max_len=3, use_colnames=True)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [80]:
is_ap.sort_values(by = ['support'], ascending = False)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,support,itemsets
4,0.178956,(비만문답)
19,0.150373,(플로차트 정형외과 진단)
11,0.077879,(윤상훈·권병조의 알짜 근육학)
23,0.060066,(흔히보는 정형외과 외래진료 가이드북)
10,0.048467,(외래에서 꼭 알아야 할 통증증후군 137가지)
22,0.045568,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼)
21,0.043911,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록)
25,0.037283,"(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼, 황황교수의 개원 한의사를 위한 상한금궤 처방 강의록)"
14,0.035626,(초음파 유도하 침 시술 가이드북)
6,0.033969,(사암침의 해석과 임상)


# Calculate Association Rules

In [81]:
from mlxtend.frequent_patterns import association_rules

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [82]:
rules_confidence = association_rules(is_ap, min_threshold=0.01) # default가 confidence
rules_lift = association_rules(is_ap, metric= "lift", min_threshold=0.01)
display(rules_lift.tail())
# display(rules_confidence)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(비만문답),(사암침의 해석과 임상),0.178956,0.033969,0.026512,0.148148,4.361337,0.020433,1.134037
1,(사암침의 해석과 임상),(비만문답),0.033969,0.178956,0.026512,0.780488,4.361337,0.020433,3.740311
2,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.045568,0.043911,0.037283,0.818182,18.632933,0.035282,5.258492
3,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.043911,0.045568,0.037283,0.849057,18.632933,0.035282,6.323115


In [83]:
rules_confidence_item = association_rules(is_ap, min) # default가 confidence
# rules_lift = association_rules(is_ap, metric= "lift", min_threshold=0.01)
# display(rules_lift)
display(rules_confidence_item)

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: Metric must be 'confidence' or 'lift', got '<built-in function min>'

In [84]:
display(rules_lift.sort_values(by=['support'], ascending = False).head(10))
display(rules_confidence.sort_values(by=['support'], ascending = False).head(10))


/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.045568,0.043911,0.037283,0.818182,18.632933,0.035282,5.258492
3,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.043911,0.045568,0.037283,0.849057,18.632933,0.035282,6.323115
0,(비만문답),(사암침의 해석과 임상),0.178956,0.033969,0.026512,0.148148,4.361337,0.020433,1.134037
1,(사암침의 해석과 임상),(비만문답),0.033969,0.178956,0.026512,0.780488,4.361337,0.020433,3.740311


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.045568,0.043911,0.037283,0.818182,18.632933,0.035282,5.258492
3,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.043911,0.045568,0.037283,0.849057,18.632933,0.035282,6.323115
0,(비만문답),(사암침의 해석과 임상),0.178956,0.033969,0.026512,0.148148,4.361337,0.020433,1.134037
1,(사암침의 해석과 임상),(비만문답),0.033969,0.178956,0.026512,0.780488,4.361337,0.020433,3.740311


In [85]:
#  we can sort the association rules according to leverage value and find the most positively correlated item sets.
display(rules_lift.sort_values(by=['conviction'], ascending = False))

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
3,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.043911,0.045568,0.037283,0.849057,18.632933,0.035282,6.323115
2,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.045568,0.043911,0.037283,0.818182,18.632933,0.035282,5.258492
1,(사암침의 해석과 임상),(비만문답),0.033969,0.178956,0.026512,0.780488,4.361337,0.020433,3.740311
0,(비만문답),(사암침의 해석과 임상),0.178956,0.033969,0.026512,0.148148,4.361337,0.020433,1.134037


# (1) Recommend Most Often Previously Purchased Items

In [86]:
tmp = train.groupby(['customer_id','name_x'])['date_paid'].agg('count').reset_index()
tmp.columns = ['customer_id','name_x','ct']
train = train.merge(tmp,on=['customer_id','name_x'],how='left')
train = train.sort_values(['ct','date_paid'],ascending=False)
train = train.drop_duplicates(['customer_id','name_x'])
train = train.sort_values(['ct','name_x'],ascending=False)
train

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,date_paid,customer_id,name_x,product_ids,max_dat,diff_dat,ct
2337,2022-07-28,5f6887a448060c52caf148ba,플로차트 정형외과 진단,62bbee3a08e04900234e36c8,2022-07-28,0,2
1831,2022-07-21,5eeb1a92e510ee503167cf7f,비만문답,629860599d93880024071acc,2022-07-25,4,2
2848,2022-08-12,5e3455f64267e105dfff116b,근육학,62551d64cb28e6002469d8f9,2022-08-12,0,2
3101,2022-08-21,5db2e8840dabe405b156f50e,KCD 한방내과 진찰진단 가이드라인,5d13115e32026c0b35383897,2022-08-21,0,2
3110,2022-08-21,5e997e27bfe4260944785d49,흔히보는 정형외과 외래진료 가이드북,6182113bbc63410023473754,2022-08-21,0,1
...,...,...,...,...,...,...,...
1263,2022-07-11,5d64953c4e77525ec5ca16bf,2022 통장잔고 2배 늘려주는 병의원 절세비법,6279c9439d93880024062730,2022-07-11,0,1
704,2022-06-21,5d96981e0dabe405b156da2c,2022 통장잔고 2배 늘려주는 병의원 절세비법,6279c9439d93880024062730,2022-06-21,0,1
195,2022-05-24,5ea8cbe41a3ca36b96784ddf,2022 통장잔고 2배 늘려주는 병의원 절세비법,6279c9439d93880024062730,2022-05-24,0,1
144,2022-05-23,5d6696914e77525ec5ca17b9,2022 통장잔고 2배 늘려주는 병의원 절세비법,6279c9439d93880024062730,2022-05-23,0,1


# (2) Recommend Items Purchased Together

In [94]:
is_ap = apriori(orders_1hot, min_support=0.01, use_colnames=True)
rules_confidence_item = association_rules(is_ap, min_threshold=0.5) # default가 confidence
rules_confidence_item

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(사암침의 해석과 임상),(비만문답),0.033969,0.178956,0.026512,0.780488,4.361337,0.020433,3.740311
1,(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),0.045568,0.043911,0.037283,0.818182,18.632933,0.035282,5.258492
2,(황황교수의 개원 한의사를 위한 상한금궤 처방 강의록),(황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼),0.043911,0.045568,0.037283,0.849057,18.632933,0.035282,6.323115


In [114]:
apriori_items = rules_confidence_item[['antecedents','consequents']].iloc[:15]

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [115]:
apriori_items['antecedents'] = apriori_items['antecedents'].apply(lambda x: list(map(str,x))[0])
apriori_items['consequents'] = apriori_items['consequents'].apply(lambda x: list(map(str,x))[0])

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [120]:
apriori_items.rename(columns={"antecedents": "A", "consequents": "A의 연관 아이템 (B)"})

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,A,A의 연관 아이템 (B)
0,사암침의 해석과 임상,비만문답
1,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록
2,황황교수의 개원 한의사를 위한 상한금궤 처방 강의록,황황교수의 임상의를 위한 근거기반 상한금궤 처방 매뉴얼


In [99]:
train_items = train[['date_paid','customer_id','name_x','product_ids']]

# apriori_items
# train_items[train_items['name_x']==]
train_items['apriori_pred'] = train_items['name_x'].apply(lambda x: apriori_items[apriori_items['antecedents']==x]['consequents'].values[0] if x in list(apriori_items['antecedents']) \
                            else None)
# for idx, row in train_items.iterrows():

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [100]:
name_x_df = train_items.groupby('customer_id')['name_x'].unique().reset_index()
apriori_pred_df  = train_items.groupby('customer_id')['apriori_pred'].unique().reset_index()
real_pred = test.groupby('customer_id')['name_x'].unique().reset_index().rename(columns={'name_x':'real_product'})

train_pred = pd.merge(name_x_df,apriori_pred_df, on='customer_id', how='left')
train_real_pred = pd.merge(train_pred,real_pred, on='customer_id', how='right')

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
train_real_pred

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,customer_id,name_x,apriori_pred,real_product
0,5d61fb634e77525ec5ca1585,[플로차트 정형외과 진단],[None],"[미안침, 파킨슨병 한의진료, 필라테스 해부학(PILATES ANATOMY)]"
1,5d6f735b19efa30eb2914094,[약침의 정석 –통증편],[None],"[뇌의학의 첫걸음, 한의학 상담, 수면장애의 한방치료]"
2,5d83468b3f0e6805c47067c9,[외래에서 꼭 알아야 할 통증증후군 137가지],[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 비만문답]"
3,5d9690200dabe405b156d8c0,[그린만의 수기의학 원리],[None],"[임상 한의사를 위한 기본 한약처방 강의 2판, 플로차트 정형외과 진단, 장골의 PI 변위는 없다]"
4,5d96fccb0dabe405b156dbc9,[플로차트 정형외과 진단],[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 카이로프랙틱 기본테크닉론]"
5,5d9708790dabe405b156dccc,"[파킨슨병 한의진료, 이해하면 보이는 어깨치료 ABC, 우리 아이 감기, 비수술정형외과, 무릎통증(아프니까 무릎이다)]",[None],"[감별진단의 정석, 신경학 증상의 감별법, The 정형내과(The Orthopaedic Medicine), 초음파 유도하 침 시술 가이드북, 장골의 PI 변위는 없다]"
6,5d970b4f0dabe405b156dce4,"[SMART 응급진료매뉴얼(세트), SMART 소아진료매뉴얼 3판, SMART 기본 일차진료매뉴얼 3판(세트)]",[None],"[초음파 유도하 침 시술 가이드북, 영어 진료 가이드북 , 소아피부질환해설]"
7,5d9c0beb0dabe405b156e1de,"[이것이 알고싶다! 당뇨병진료, 그림으로 이해하는 수분과 전해질 및 산-염기 대사, 갑상선 진료 완전정복]",[None],"[초음파 유도하 침 시술 가이드북, SMART 응급진료매뉴얼(세트), 영어 진료 가이드북 ]"
8,5dce69512bb59605ca3cffb6,"[한의 피부진료 첫 걸음, 침의 과학적 접근의 이해, 말초신경 약침의학, 근골격계 약침의학]",[None],"[임상 한의사를 위한 기본 한약처방 강의 2판, 일차진료 한의사를 위한 보험한약입문 - 둘째 판, 숲을 보는 요통치료, NEO 인턴 핸드북]"
9,5df191784267e105dffef5d1,[초음파 유도하 침 시술 가이드북],[None],"[비만문답, 바른 자세를 위한 교정운동, 약침의 정석 –통증편]"


In [102]:
# 아이템 맞춘 개수
cnt = 0
for preds, reals in zip(train_real_pred['apriori_pred'], train_real_pred['real_product']):
    for pred in preds:
        if pred in reals:
            cnt += 1
cnt

/home/user_4/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0